In [1]:
import pickle
import time
import datetime

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np

import cudf
import cupy


# Load in Data

a set of wells from the permian basin are included in parquet format (`wells_permian.parquet`) There are 383970 wells! 

If you have already loaded well data into OmniSci, you can easily read it by using pymapd (https://pymapd.readthedocs.io/en/latest/)

Note: Loading in the data may take almost a minute


In [2]:
omnisci=False
if omnisci:
    from pymapd import connect

    con = connect(user="abraham", 
                         password="abraham", 
                         host="localhost", 
                         dbname="abraham", 
                         port=6273,
                         protocol='http')

    print(con)

    # if loading in from omnisci:
    l_cols = ['wellapi',
     'state',
     'formationname',
     'subbasin',
     'operator',
     'firstproducingmonth',
     'ProductionPeriodStartDate',
     'ProducingDaysInPeriod',
     'SequentialMonthWell',
     'OilProduced',
     'GasProduced',
     'WaterProduced',
     'CondensateProduced',
     'BOE']
    s_cols = (', ').join(l_cols)

    query = "SELECT " + s_cols + " FROM well WHERE majorbasin='Permian' LIMIT 800000000"
    print(query)

    df_permian = pd.read_sql(query,con)
    df_permian.head()

In [ ]:
parquet_file = 'wells_permian.parquet'
t0 = time.time()
df_permian = pd.read_parquet(parquet_file, engine='pyarrow')
print("Time to read from parquet: ", time.time()-t0)

In [3]:
df_permian.head()

NameError: name 'df_permian' is not defined

In [ ]:
start_date = datetime.datetime(2011,1,1)

end_date = datetime.datetime(2015,1,1)

no_wells = len(df_permian['wellapi']
            .loc[(df_permian['firstproducingmonth'] > start_date) & (df_permian['firstproducingmonth'] <= end_date)]
            .unique()
           )
print("Number of wells starting production from %s: %d" %(start_date.year, no_wells))

In [ ]:
production_col = 'BOE'

df_permian_recent = (df_permian.loc[df_permian['firstproducingmonth'] > start_date]
                     .sort_values(production_col, ascending=False)
                     .drop_duplicates(['wellapi','SequentialMonthWell'])
                     .dropna(subset=[production_col, 'SequentialMonthWell'])
                     .sort_values(['wellapi',production_col],ascending=True)
                 )

well_information = (df_permian_recent[['wellapi','SequentialMonthWell']]
 .groupby(['wellapi'])['SequentialMonthWell']
 .count()
 .reset_index(name='count')
 .sort_values(['count'], ascending=False)
)

well_information.head()

## Quickly viewing n number of random wells

In [ ]:
# plot n number of wells together to get an idea of what the production looks like
n = 50

fig, ax = plt.subplots()
#well_apis = df_permian_recent['wellapi'].sample(n)
well_apis = well_information['wellapi'].head(n)
for well_api in well_apis:
    
    well = df_permian_recent.loc[df_permian_recent['wellapi']==well_api].sort_values('SequentialMonthWell',ascending=True)
    ax.plot(well['SequentialMonthWell'],well[production_col])

ax.set(xlabel='Months in production', ylabel='Production '+production_col,
       title='Production')
ax.grid()

plt.show()

# Looking at one well at random

In [ ]:
api = np.random.choice(df_permian_recent['wellapi'].unique())
one_well = df_permian_recent[df_permian_recent['wellapi']==api].sort_values('SequentialMonthWell',ascending=True)


In [ ]:
# Data for plotting
fig, ax = plt.subplots()
ax.plot(one_well.SequentialMonthWell, one_well[production_col])

ax.set(xlabel='Months in production', ylabel='Production',
       title='Production')
ax.grid()

plt.show()

# Decline curve analysis

In [ ]:
import arps

qi = arps.max_qi(df=one_well, month_col='SequentialMonthWell', interval=4, prod=production_col)


popt_exp, pcov_exp = curve_fit(arps.exponential, one_well['SequentialMonthWell'], one_well[production_col],bounds=(0, [qi,20]))

popt_hyp, pcov_hyp = curve_fit(arps.hyperbolic, one_well['SequentialMonthWell'], 
                                    one_well[production_col],bounds=(0, [qi,2,20]))

qi = popt_exp[1]
b = popt_hyp[1]
di_exp = popt_exp[1]
di_hyp = popt_hyp[2]
print('Exponential Fit Curve-fitted Variables: qi=%f, di=%f' %(popt_exp[0], popt_exp[1]) )
print('Hyperbolic Fit Curve-fitted Variables: qi=%f, b=%f, di=%f' %(popt_hyp[0], popt_hyp[1], popt_hyp[2]))



one_well.loc[:,'exponential_predicted'] = arps.exponential(one_well['SequentialMonthWell'],*popt_exp)
one_well.loc[:,'hyperbolic_predicted'] = arps.hyperbolic(one_well['SequentialMonthWell'],*popt_hyp)


In [ ]:
def plot_actual_vs_predicted_by_equations(df, x_variable, y_variables, plot_title):
    """
    This function is used to map x- and y-variables against each other
    Arguments:
        df: Pandas dataframe.
        x_variable: String. Name of the column that we want to set as the 
        x-variable in the plot
        y_variables: string (single), or list of strings (multiple). Name(s) 
        of the column(s) that we want to set as the y-variable in the plot
    """
    #Plot results
    df.plot(x=x_variable, y=y_variables, title=plot_title)
    plt.show()

In [ ]:
y_variables=[production_col, "hyperbolic_predicted", "exponential_predicted"]
x_variable='SequentialMonthWell'
#Create the plot title
plot_title=production_col+' Production for one well'
#Plot the data to visualize the equation fit
plot_actual_vs_predicted_by_equations(one_well, x_variable, y_variables, plot_title)


# Calculating the values for all wells

In [ ]:
# # import arps
# grouped = df_permian_recent.groupby('wellapi')

# well_data = []

# t0 = time.time()
# for (name, well) in grouped:
    
    
#     try:
#         well = well.sort_values('SequentialMonthWell',ascending=True)
#         qi = arps.max_qi(df=well, month_col='SequentialMonthWell', interval=6, prod=production_col)

#         popt_exp, pcov_exp = curve_fit(arps.exponential, well['SequentialMonthWell'], well[production_col],bounds=(0, [qi,20]))
#         popt_hyp, pcov_hyp = curve_fit(arps.hyperbolic, well['SequentialMonthWell'], 
#                                             well[production_col],bounds=(0, [qi,2,20]))

#         print("Well API: ", name)
#         print('Exponential: qi=%f, di=%f' %(popt_exp[0], popt_exp[1]) )
#         print('Hyperbolic: qi=%f, b=%f, di=%f' %(popt_hyp[0], popt_hyp[1], popt_hyp[2]))
#         print('\n')

#         qi = popt_exp[1]
#         b = popt_hyp[1]
#         di_exp = popt_exp[1]
#         di_hyp = popt_hyp[2]

#         well_data.append((name, qi, b, di_exp, di_hyp))
#     except:
#         print("Well %s not forecasted: %f" %(name, qi))
# print("Time: ", time.time()-t0)    

In [ ]:
def f(x):
    try: 
        name = x[0]
        well = x[1]

        well = well.sort_values('SequentialMonthWell',ascending=True)
        qi = arps.max_qi(df=well, month_col='SequentialMonthWell', interval=6, prod=production_col)

        popt_exp, pcov_exp = curve_fit(arps.exponential, well['SequentialMonthWell'], well[production_col],bounds=(0, [qi,20]))
        popt_hyp, pcov_hyp = curve_fit(arps.hyperbolic, well['SequentialMonthWell'], 
                                            well[production_col],bounds=(0, [qi,2,20]))

#         print("Well API: ", name)
#         print('Exponential: qi=%f, di=%f' %(popt_exp[0], popt_exp[1]) )
#         print('Hyperbolic: qi=%f, b=%f, di=%f' %(popt_hyp[0], popt_hyp[1], popt_hyp[2]))
#         print('\n')

#         qi = popt_exp[1]
#         b = popt_hyp[1]
#         di_exp = popt_exp[1]
#         di_hyp = popt_hyp[2]
        # name, qi, b, di_exp, di_hyp
        return (name, popt_exp[1], popt_hyp[1], popt_exp[1], popt_hyp[2])
    except:
        print("Well %s not forecasted: %f" %(name, qi))

In [ ]:
from multiprocessing import Pool
grouped = df_permian_recent.groupby('wellapi')
p = Pool()
values = p.map(f, [x for x in grouped])
values = [v for v in values if v is not None]

In [7]:
values = [v for v in values if v is not None]

In [3]:
import pickle

values = pickle.load(open('../data/values.p','rb'))

In [4]:
from pymapd import connect

con = connect(user="abraham", 
                     password="abraham", 
                     host="localhost", 
                     dbname="abraham", 
                     port=6274,
                     protocol='binary')

ERROR:Could not connect to any of [('127.0.0.1', 6274)]


OperationalError: Could not connect to database

In [ ]:
def power(x, y): 
    # Initialize result 
    res = 1
    while (y > 0): 
        # If y is odd, multiply 
        # x with result 
        if ((y & 1) == 1) : 
            res = res * x 
        # n must be even  
        # now y = y/2 
        y = y >> 1
        # Change x to x^2 
        x = x * x 
    return res 

In [11]:
import numpy as np
@con('float32(int32, float32, float32, float32)')
def hyperbolic(t, qi, b, di):
    def power(x, y): 
        if (y == 0): return 1
        elif (int(y % 2) == 0): 
            return (power(x, int(y / 2)) *
                   power(x, int(y / 2))) 
        else: 
            return (x * power(x, int(y / 2)) *
                       power(x, int(y / 2))) 
        
    return qi/(power( (1.0+b*di*t), (1.0/b) ) )



In [10]:
#include <stdio.h>
#include <math.h>

def power():
    

int main()
{
double base, expo, res;
printf("Enter a base number: ");
scanf("%lf", &base);
printf("Enter an exponent: ");
scanf("%lf", &expo);
res = pow(base, expo);
printf("%.1lf^%.1lf = %.2lf", base, expo, res);
return 0;
}

In [ ]:
while y > 0:
    if (int(y % 2) == 0):
        

In [7]:
import numpy as np
@con('float32(int32, float32, float32)')
def exponential(t, qi, di):
    return qi*np.exp((-di*t))

In [12]:
import rbc

In [7]:
import numpy as np
@con('double(double)')
def add_two(a):
    return np.add(a,2.0)

/home/mapdadmin/miniconda3/envs/decline_curve/lib/python3.7/site-packages/rbc/remotejit.py:239: UserWarning: <ipython-input-7-cc9bcf7e659d>#2 re-implements <ipython-input-6-3537560d3e79>#2 for `float64(float64)`
  .format(f1=f1, n1=n1, f2=f2, n2=n2, sig=sig))


In [8]:
con.get_table_details('well_parameters')

[ColumnDetails(name='wellapi', type='STR', nullable=True, precision=0, scale=0, comp_param=32, encoding='DICT', is_array=False),
 ColumnDetails(name='qi_hyp', type='DOUBLE', nullable=True, precision=0, scale=0, comp_param=0, encoding='NONE', is_array=False),
 ColumnDetails(name='b', type='DOUBLE', nullable=True, precision=0, scale=0, comp_param=0, encoding='NONE', is_array=False),
 ColumnDetails(name='di_exp', type='DOUBLE', nullable=True, precision=0, scale=0, comp_param=0, encoding='NONE', is_array=False),
 ColumnDetails(name='di_hyp', type='DOUBLE', nullable=True, precision=0, scale=0, comp_param=0, encoding='NONE', is_array=False)]

In [9]:
query = ("Select add_two(di_hyp) from well_parameters limit 10")

df = con.select_ipc(query)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Use of unsupported NumPy function 'numpy.add' or unsupported use of the function.

File "<ipython-input-7-cc9bcf7e659d>", line 4:
def add_two(a):
    return np.add(a,2.0)
    ^

[1] During: typing of get attribute at <ipython-input-7-cc9bcf7e659d> (4)

File "<ipython-input-7-cc9bcf7e659d>", line 4:
def add_two(a):
    return np.add(a,2.0)
    ^


In [ ]:
# name, qi, b, di_exp, di_hyp
# df_parameters = pd.DataFrame(values, columns=['wellapi','b','qi_exp','qi_hyp','di_exp','di_hyp'])
df_parameters = pd.DataFrame(values, columns=['wellapi','qi_hyp','b','di_exp','di_hyp'])
df_parameters = df_parameters.dropna()

In [ ]:
name = "well_parameters"
con.load_table(name, df_parameters)

In [ ]:
df = pd.read_sql("SELECT * from well_parameters",con)

In [ ]:
query = "SELECT w.*, hyperbolic(w.SequentialMonthWell, p.qi_hyp, p.b, p.di_hyp) FROM well w, well_parameters p WHERE w.wellapi = p.wellapi limit 100"

df = pd.read_sql(query,con)

In [ ]:
SELECT w.*, hyperbolic(w.SequentialMonthWell, p.qi_hyp, p.b, p.di_hyp) 
FROM well w, parameters p
WHERE w.wellapi = p.wellapi